In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import os
import numpy as np
import seaborn as sb
import matplotlib.pyplot as plt
import pandas as pd
import missingno as msg
from scipy import stats
from sklearn.model_selection import train_test_split , cross_val_score , RandomizedSearchCV
from sklearn.preprocessing import StandardScaler ,MinMaxScaler ,LabelEncoder
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier

In [ ]:
df = pd.read_csv("/kaggle/input/titanic/train.csv")
df1 = pd.read_csv("/kaggle/input/titanic/test.csv")
sub = pd.read_csv("/kaggle/input/titanic/gender_submission.csv")


In [ ]:
df

In [ ]:
msg.matrix(df)

In [ ]:
msg.matrix(df1)

# *We will first Analysis Null Values for all the Column .*

## Analysis on Age col

In [ ]:
sb.kdeplot(df['Age'])

In [ ]:
sb.distplot(df['Age'])

In [ ]:
print("Skewness of Age is " , stats.skew(df['Age'].dropna()))
print("Kurtosis of Age is " , stats.kurtosis(df['Age'].dropna()))


In [ ]:
df.corr()['Age'].sort_values(ascending = False)

In [ ]:
print(df['Age'].isna().sum())
print(df1['Age'].isna().sum())


In [ ]:
sb.boxplot(df['Age'])

In [ ]:
sb.boxplot(df1['Age'])

In [ ]:
df['Age'].fillna(df['Age'].median() , inplace = True)
df1['Age'].fillna(df1['Age'].median() , inplace = True)


In [ ]:
sb.distplot(df['Age'] , label = "train_age" , color = "red")
sb.distplot(df1['Age'] , label = "test_age" , color = "blue")


## Analysis on Fare col

In [ ]:
sb.distplot(df['Fare'])

In [ ]:
sb.boxplot(df['Fare'])

In [ ]:
sb.distplot(df1['Fare'])

In [ ]:
sb.boxplot(df1['Fare'])

In [ ]:
print("Skewness of Fare is " , stats.skew(df1['Fare'].dropna()))
print("Kurtosis of Fare is " , stats.kurtosis(df1['Fare'].dropna()))


In [ ]:
print(df['Fare'].value_counts().sort_values(ascending = True))

In [ ]:
print(df1['Fare'].value_counts().sort_values(ascending = True))

In [ ]:
df = df[df['Fare'] < 100]    #remove outlier

In [ ]:
sb.distplot(df['Fare'])

In [ ]:
print("Skewness of Fare is " , stats.skew(df['Fare'].dropna()))
print("Kurtosis of Fare is " , stats.kurtosis(df['Fare'].dropna()))


In [ ]:
print(df['Fare'].isna().sum())

In [ ]:
df1['Fare'].fillna(df1['Fare'].median() , inplace = True)

## Analysis on Cabin col

In [ ]:
df['Cabin']

In [ ]:
df['Cabin'].unique()

In [ ]:
print((df['Cabin'].isna().sum() / len(df))*100)

In [ ]:
# drop
df.drop(['Cabin'],axis =1 , inplace = True)
df1.drop(['Cabin'],axis =1 , inplace = True)


In [ ]:
df.head()

## Analysis on Embarked col

In [ ]:
df['Embarked']

In [ ]:
print(df['Embarked'].unique())
print(df['Embarked'].nunique())


In [ ]:
df['Embarked'].fillna(method = 'ffill' , inplace = True)

In [ ]:
df['Embarked'].isna().sum()

In [ ]:
df1['Embarked'].fillna(method = 'ffill' , inplace = True)

# *Let's do Feature Engineering (EDA)*

In [ ]:
df.head()

In [ ]:
df.drop(['PassengerId'],axis =1 , inplace =True)
df1.drop(['PassengerId'],axis =1 , inplace =True)


## Analysis on Name col

In [ ]:
df['Name']

In [ ]:
df = df.reset_index(drop = True)
for i in range(len(df)):
    if ("Mr" in df.loc[i , 'Name']) or ("Master" in df.loc[i , 'Name']):
        df.loc[i , "gender_category"] = 0
    else:
        df.loc[i , 'gender_category'] = 1

In [ ]:
#df = df.reset_index(drop = True)
for i in range(len(df1)):
    if ("Mr" in df1.loc[i , 'Name']) or ("Master" in df1.loc[i , 'Name']):
        df1.loc[i , "gender_category"] = 0
    else:
        df1.loc[i , 'gender_category'] = 1

In [ ]:
df.drop(['Name'],axis =1 , inplace = True)
df1.drop(['Name'],axis =1 , inplace = True)


## Analysis on Sex

In [ ]:
df['Sex']

In [ ]:
sb.countplot(df['Sex'])

In [ ]:
sb.countplot(df1['Sex'])

In [ ]:
le = LabelEncoder()
df['Sex'] = le.fit_transform(df['Sex'])
df1['Sex'] = le.fit_transform(df1['Sex'])


In [ ]:
df['Sex'].unique()

## Analysis on Ticket

In [ ]:
df['Ticket']

In [ ]:
df.drop(['Ticket'],axis =1 , inplace = True)
df1.drop(['Ticket'],axis =1 , inplace = True)


## Analysis on Embarked

In [ ]:
sb.countplot(df['Embarked'])

In [ ]:
sb.countplot(df1['Embarked'])

In [ ]:
df['Embarked'] = df['Embarked'].replace({"S":3 , "C":2 , "Q":1})
df1['Embarked'] = df1['Embarked'].replace({"S":3 , "C":2 , "Q":1})


In [ ]:
df.head()

In [ ]:
df1.head()

In [ ]:
plt.figure(figsize = (10,8))
sb.heatmap(df.corr() , annot = True)

In [ ]:
df.corr()['Survived'].sort_values(ascending = False)

# Splitting , Cross Validation and Model Building

In [ ]:
x = df.drop(['Survived'] ,axis =1)
y = df['Survived']

In [ ]:
x_train ,x_test , y_train , y_test = train_test_split(x,y,test_size= 0.27 , random_state =100)

In [ ]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)


## Model - RandomForest

In [ ]:
rf = RandomForestClassifier()

In [ ]:
result = cross_val_score(rf ,x,y, cv =10 ,scoring = 'accuracy',n_jobs =-1 , verbose =1)

In [ ]:
print(result.mean())

In [ ]:
rf.fit(x_train , y_train)

In [ ]:
rf_pred = rf.predict(x_test)

In [ ]:
print("Acc of RandomForest without HyperParameter tuning is " , accuracy_score(rf_pred , y_test))

## Model - XGBoostClassifier

In [ ]:
xgb = XGBClassifier()
result = cross_val_score(xgb ,x,y, cv =10 ,scoring = 'accuracy',n_jobs =-1 , verbose =1)

In [ ]:
print(result.mean())

In [ ]:
print(xgb.fit(x_train , y_train))
xgb_pred = xgb.predict(x_test)


In [ ]:
print("Acc of XGBClassifier without HyperParameter tuning is " , accuracy_score(xgb_pred , y_test))

## HyperParam tuning for RandomForest

In [ ]:
rf_param = {'n_estimators':list(range(100,500)) , 
         'max_depth':list(range(1,10)) , 
         'criterion':['gini','entropy'] ,
         'max_samples':list(range(1,10))    
}

In [ ]:
rscv = RandomizedSearchCV(rf ,param_distributions=rf_param ,  cv =5 , n_iter=10 , scoring = 'accuracy',n_jobs =-1 , verbose =10)

In [ ]:
rscv.fit(x,y)

In [ ]:
print(rscv.best_score_)
print(rscv.best_estimator_)
print(rscv.best_index_)
print(rscv.best_params_)


In [ ]:
# Fit and predict
rf = RandomForestClassifier(criterion='entropy', max_depth=3, max_samples=9,
                       n_estimators=119)
print(rf.fit(x_train , y_train))
print("Accuracy score of RF after tuning is :", accuracy_score(rf.predict(x_test) , y_test))

## HyperParam Tuning for XGBoost classifier

In [ ]:
xgb_param = {'n_estimators':list(range(100,500)) , 
         'max_depth':list(range(1,10)) , 
         'learning_rate':[0.001,0.002,0.003,0.004,0.005,0.006,0.007,0.008,0.05,0.09] ,
         'min_child_weight ':list(range(1,10))    
}

rscv = RandomizedSearchCV(xgb ,param_distributions=xgb_param ,  cv =5 , n_iter=10 , scoring = 'accuracy',n_jobs =-1 , verbose =10)
rscv.fit(x,y)

In [ ]:
print(rscv.best_score_)
print(rscv.best_estimator_)
print(rscv.best_index_)
print(rscv.best_params_)


In [ ]:
# Fit and predict
xgb = XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.09, max_delta_step=0, max_depth=2,
               min_child_weight =9,
              monotone_constraints='()', n_estimators=448, n_jobs=0,
              num_parallel_tree=1, random_state=0, reg_alpha=0, reg_lambda=1,
              scale_pos_weight=1, subsample=1, tree_method='exact',
              validate_parameters=1, verbosity=None)

print(xgb.fit(x_train , y_train))
print("Accuracy score of XGB after tuning is :", accuracy_score(xgb.predict(x_test) , y_test))

## Acc of RF - 75.33
## Acc of XGB - 83.2

### Final Model selected - XGB

In [ ]:
df1

In [ ]:
main_pred = xgb.predict(df1)

In [ ]:
main_pred

## Make Submission file

In [ ]:
sub

In [ ]:
sub['Survived'] = main_pred

In [ ]:
sub.to_csv("Main_Submission.csv" , index = False)

## If you liked this Notebook , please upvote. Gives  Motivation to make new Notebooks :)